# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,northam,51.0333,-4.2167,8.12,89,100,4.06,GB,1672625158
1,1,micheweni,-4.9667,39.8333,26.34,76,8,3.65,TZ,1672625466
2,2,jamestown,42.0970,-79.2353,5.57,75,100,3.60,US,1672625178
3,3,anadyr,64.7500,177.4833,-34.73,98,93,3.82,RU,1672625020
4,4,pavullo nel frignano,44.3315,10.8350,9.32,86,84,1.51,IT,1672625469


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map plot
map_cities



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_max = 30
ideal_temp_min = 25
ideal_wind = 3
ideal_humidity = 85
ideal_cities=[]
nogo_cities=[]

for index, row in city_data_df.iterrows():
        temp = row["Max Temp"]
        wind = row["Wind Speed"]
        humidity = row["Humidity"]
        city_ideal = row["City"]
        if temp > ideal_temp_min:
            if temp < ideal_temp_max:
                if wind < ideal_wind:
                    if humidity < ideal_humidity:
                        ideal_cities.append(row["City"])
                    # print (f"{city_ideal} is ideal" )
        else:
            nogo_cities.append(row["City"])
            # print (f"{city_ideal} is not somehwere I want to go" )


ideal_cities_df = pd.DataFrame(ideal_cities, columns =["City"])
ideal_cities_alldf = pd.merge(ideal_cities_df, city_data_df, on=["City"], how ="left")        
# Drop any rows with null values
ideal_cities_alldf = ideal_cities_alldf.dropna()

# Display sample data
ideal_cities_alldf = pd.merge(ideal_cities_df, city_data_df, on=["City"], how ="left")
ideal_cities_alldf.sample(5)

,City,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,madimba,320,-10.4000,40.3333,25.17,82,90,0.61,TZ,1672625784
5,lorengau,121,-2.0226,147.2712,29.49,69,42,2.59,PG,1672625585
6,prabumulih,125,-3.4500,104.2500,26.04,83,100,1.82,ID,1672625589
24,namatanai,462,-3.6667,152.4333,29.62,66,99,2.05,PG,1672625926
14,ocos,258,14.5094,-92.1933,26.10,76,100,2.91,GT,1672625725


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_alldf.loc[:,["City", "Country", "Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"]=""

# Display sample data
hotel_df.sample(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
1,sibolga,ID,1.7427,98.7792,82,
26,bengkulu,ID,-3.8004,102.2655,76,
5,lorengau,PG,-2.0226,147.2712,69,
21,tabuk,PH,17.4189,121.4443,80,
32,san matias,BO,-16.3667,-58.4000,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
radius = 5000
categories = "accommodation.hotel"
language = "fr"
limit = 20

    # YOUR CODE HERE
params = {"radius":radius,
            "categories":categories,
            "limit":limit,
            "apiKey":geoapify_key}

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    srch_lat = row["Lat"]
    srch_lon = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary


    params["filter"] = f"circle:{srch_lon},{srch_lat},{radius}"
    params["bias"] = f"proximity:{srch_lon},{srch_lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    name_address = requests.get(base_url, params=params).json()

    print(name_address["features"][0]["properties"]["name"])

Woodbridge on the Derwent
Hotel Prima Indah


IndexError: list index out of range

In [ ]:
# Set parameters to search for a hotel
radius = 5000
categories = "accommodation.hotel"
limit = 20

    # YOUR CODE HERE
params = {"radius":radius,
            "categories":categories,
            "limit":limit,
            "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    srch_lat = row["Lat"]
    srch_lon = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary


    params["filter"] = f"circle:{srch_lon},{srch_lat},{radius}"
    params["bias"] = f"proximity:{srch_lon},{srch_lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
# print(json.dumps(name_address, indent=4, sort_keys=True))

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_vacation= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols=["City","Country","Hotel Name","Lat","Lng","Humidity"])
    # hover_cols=["City","Country","Hotel Name","Lat","Lng","Humidity"])


# Display the map
map_vacation


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)